# Audience media и рекламы
Пример отчета по расчету совокупной аудитории интернет-ресурсов (Сross Web) и профиля рекламных кампаний в Интернете.

## Описание задачи

Посчитаем аудиторию по ресурсу и рекламным сетям по всем типам пользования интернетом:

- совокупную аудиторию ресурса Вконтакте и рекламных сетей Everest-network и Betweenx;
- эксклюзивную аудиторию ресурса Вконтакте и рекламных сетей Everest-network и Betweenx; 
- совокупную аудиторию ресурса Вконтакте и рекламных сетей Everest-network и Betweenx с разбивкой по социально-демографической переменной "Пол".

Общие параметры:
- Период: Октябрь 2023
- География: Россия 0+
- Население: 12+
- Тип пользования интернетом: ограничения нет, считаем по всем (Web Desktop, Web Mobile, App Mobile)

Статистики:
- Reach (reach)
- Reach% (reachPer)
- Reach N+ Distribution (reachN)
- Opportunity To See (ots)
- Exclusive Reach (exclusiveReach)

# Инициализация

При построении отчета первый шаг в любом ноутбуке - загрузка библиотек, которые помогут обращаться к API Cross Web и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter

In [ ]:
%reload_ext autoreload
%autoreload 2

import pandas as pd

from mediascope_api.core import net as mscore
from mediascope_api.crossweb import tasks as cwt
from mediascope_api.crossweb import catalogs as cwc

# Настраиваем отображение

# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем максимальное количество выводимых строк. Раскомментируйте нужную строку
# 200 строк
# pd.set_option("display.max_rows", 200)
# Отображаем все строки. ВАЖНО! Отображение большого DataFrame требует много ресурсов
# pd.set_option("display.max_rows", None)

# Cоздаем объекты для работы с API Cross Web
mnet = mscore.MediascopeApiNetwork()
mtask = cwt.CrossWebTask()
cats = cwc.CrossWebCats()

## Формируем задания

Для расчета статистик со шкалами обязательно нужно указать эти шкалы в переменной **scales**, а также прописать ее в параметрах, передаваемых в функцию **build_task**.

### Общие параметры для заданий

Для начала зададим общие параметры

In [ ]:
# Задаем период
# Период указывается в виде списка ('Начало', 'Конец'). Можно указать несколько периодов
date_filter = [('2023-10-01', '2023-10-31')]

# Задаем фильтр по типам пользования интернетом для интернет-ресурсов и для рекламной сети отдельно.
media_usetype_filter = [1,2,3]
profile_usetype_filter = [1,2,3]

### Получим ID интернет-ресурса и ID рекламных сетей.
Для построения отчета необходимо получить идентификаторы ресурса __Вконтакте__ и рекламных сетей __Everest-network__ и __Betweenx__.

Для этого воспользуемся методами поиска в медиа справочнике. Работа с медиа справочником подробно описана в ноутбуке [catalogs](catalogs.ipynb). 

Получим идентификатор ресурса **Вконтакте**

In [ ]:
cats.get_resource(resource='Вконтакте')

Получим идентификатор рекламной сети **Everest-network** и рекламной сети **Betweenx**

In [ ]:
cats.get_ad_agency(agency='Everest-network')

In [ ]:
cats.get_ad_agency(agency='Betweenx')

Таким образом, необходимые идентификаторы следующие:

- **Вконтакте** resourceId = 1150
- **Everest-network** advertisementAgencyId = 12983
- **Betweenx** advertisementAgencyId = 59418

## Задания

Перейдем к формированию заданий.


### Задание №1. Расчет совокупной аудитории ресурса Вконтакте и рекламных сетей Everest-network и Betweenx

#### Важно!

Статистика __ots__ рассчитывает общее количество загрузок страниц/ открытий приложений медиа объекта за указанный период в рамках заданной соц.-дем. группы. В рамках расчета media&profile суммируются значения, полученные отдельно по media и отдельно по рекламе по тем или иным типам пользования интернета.  

In [ ]:
# Задаем название для отображения в DataFrame
task_name = 'Sample media_profile'

# Задаем фильтр по географии, в нашем случае он не требуется
geo_filter = None

# Задаем фильтр по демографии
demo_filter = None

# Задаем фильтры по медиа и рекламе
media_filter = "crossMediaResourceId = 1150"
profile_filter = "advertisementAgencyId IN (12983, 59418)"

#Указываем список срезов, чтобы сформировать структуру расчета
slices = ["researchMonth"]

# Указываем список статистик для расчета
statistics = ["reach", "reachPer", 'reachN', 'ots']

scales = {'reachN':[(20, 255)]}

# Формируем задание для API Cross Web в формате JSON
task_json = mtask.build_task_media_profile('media-profile', task_name=task_name, date_filter=date_filter,
                                           media_usetype_filter=media_usetype_filter,
                                           profile_usetype_filter=profile_usetype_filter,
                                           media_filter=media_filter,
                                           profile_filter=profile_filter,
                                           geo_filter=geo_filter, demo_filter=demo_filter, 
                                           slices=slices, statistics=statistics,
                                           scales=scales)


# Отправляем задание на расчет и ждем выполнения
task = mtask.wait_task(mtask.send_media_profile_task(task_json))

# Получаем результат
df_ad = mtask.result2table(mtask.get_result(task), project_name = task_name)
df_ad

In [ ]:
df_info = mtask.get_report_info()

with pd.ExcelWriter(mtask.get_excel_filename('Sample media_profile')) as writer:
    df_ad.to_excel(writer, sheet_name='Report', index=False)
    df_info.to_excel(writer, sheet_name='Info', index=False)

### Задание №2. Расчет эксклюзивной аудитории ресурса Вконтакте и рекламных сетей Everest-network и Betweenx

#### Важно!

- Статистика __mediaExclusiveReach__ рассчитывается как эксклюзивная аудитория тех, кто проконтактировал только с текущим медиа объектом c заданного типа пользования интернетом, но не контактировал с рекламными объектами (указанными в __profile_filter__) ни с одного из типов пользования интернетом среди совокупной аудитории текущего медиа объекта и рекламных объектов (указанных в __profile_filter__).

Например, в данной задаче: `17 254 555 жителей России 0+ в возрасте 12+ зашли на ресурс ВКонтакте с десктопного или мобильного устройства (согласно ограничениям в media_usetype_filter) хотя бы один раз за октябрь 2023 года и не контактировали с рекламными сетями Everest-network и Betweenx с десктопного или мобильного устройства.`

- Статистика __profileExclusiveReach__ рассчитывается как эксклюзивная аудитория тех, кто проконтактировал только с текущим рекламным объектом c заданного типа пользования интернетом, но не контактировал с медиа объектами (указанными в __media_filter__) ни с одного из типов пользования интернетом среди совокупной аудитории текущего рекламного объекта и медиа объектов (указанных в __media_filter__).

Например, в данной задаче: `6 525 413 жителей России 0+ в возрасте 12+ контактировали с рекламной сетью Betweenx с десктопного или мобильного устройства  (согласно ограничениям в profile_usetype_filter) хотя бы один раз за октябрь 2023 года и не заходили на ресурс ВКонтакте с десктопного или мобильного устройства. `


- Статистика __exclusiveReach__ в рамках расчета media&рекламы повторяет статистики __mediaExclusiveReach__ и __profileExclusiveReach__ согласно принадлежности проекту (media или рекламе). 

Объекты media или рекламы не участвуют в расчете эксклюзивной аудитории объекта, если они из одного проекта с рассматриваемым объектом (media или рекламы).
 


In [ ]:
# Задаем название для отображения в DataFrame
task_name = 'Sample excl_media_profile'

# Задаем фильтр по географии, в нашем случае он не требуется
geo_filter = None

# Задаем фильтр по демографии
demo_filter = None

media_filter = "crossMediaResourceId = 1150"

profile_filter = "advertisementAgencyId IN (12983, 59418)"

#Указываем список срезов, чтобы сформировать структуру расчета
slices = ["researchMonth",
          "crossMediaResourceId",
          "advertisementAgencyId"] 
          
# Указываем список статистик для расчета
statistics = ["exclusiveReach", "profileExclusiveReach", "mediaExclusiveReach"]

# Формируем задание для API Cross Web в формате JSON
task_json = mtask.build_task_media_profile('media-profile', task_name=task_name, date_filter=date_filter,
                                           media_usetype_filter=media_usetype_filter,
                                           profile_usetype_filter=profile_usetype_filter,
                                           media_filter=media_filter,
                                           profile_filter=profile_filter,
                                           geo_filter=geo_filter, demo_filter=demo_filter, 
                                           slices=slices, statistics=statistics)


# Отправляем задание на расчет и ждем выполнения
task = mtask.wait_task(mtask.send_media_profile_task(task_json))

# Получаем результат
df_ad_excl = mtask.result2table(mtask.get_result(task), project_name = task_name)
df_ad_excl

## Сохраняем в Excel

In [ ]:
df_info = mtask.get_report_info()

with pd.ExcelWriter(mtask.get_excel_filename('Sample excl_media_profile')) as writer:
    df_ad_excl.to_excel(writer, sheet_name='Report', index=False)
    df_info.to_excel(writer, sheet_name='Info', index=False)

### Задание №3. Расчет совокупной аудитории ресурса Вконтакте и рекламных сетей Everest-network и Betweenx в разбивке на социально-демографические параметры

In [ ]:
# Задаем название для отображения в DataFrame
task_name = 'Sample ta_media_profile'

# Задаем фильтр по географии, в нашем случае он не требуется
geo_filter = None

# Задаем фильтр по демографии
demo_filter = None

media_filter = "crossMediaResourceId = 1150"

profile_filter = "advertisementAgencyId IN (12983, 59418)"

#Указываем список срезов, чтобы сформировать структуру расчета
slices = ["researchMonth", "sex"] 
          
# Указываем список статистик для расчета
statistics = ["reach", "reachPer"]

# Формируем задание для API Cross Web в формате JSON
task_json = mtask.build_task_media_profile('media-profile', task_name=task_name, date_filter=date_filter,
                                           media_usetype_filter=media_usetype_filter,
                                           profile_usetype_filter=profile_usetype_filter,
                                           media_filter=media_filter,
                                           profile_filter=profile_filter,
                                           geo_filter=geo_filter, demo_filter=demo_filter, 
                                           slices=slices, statistics=statistics)


# Отправляем задание на расчет и ждем выполнения
task = mtask.wait_task(mtask.send_media_profile_task(task_json))

# Получаем результат
df_ad_ta = mtask.result2table(mtask.get_result(task), project_name = task_name)
df_ad_ta

## Сохраняем в Excel

In [ ]:
df_info = mtask.get_report_info()

with pd.ExcelWriter(mtask.get_excel_filename('Sample ta_media_profile')) as writer:
    df_ad_ta.to_excel(writer, sheet_name='Report', index=False)
    df_info.to_excel(writer, sheet_name='Info', index=False)